In [ ]:
import nltk

# Demo 5B #
The purpose of this demonstration is reiterate topics covered in Lectures 5.3 and 5.4. After this demonstration, you should feel comfortable:
- implementing an K-means clustering model with `sklearn`
- evaluating features contributing to cluster assignment
- evaluating and tuning a K-means model
- using PCA for dimension reduction
- implementing an HDBSCAN model
- using UMAP for dimension reduction

## The Data ##

We're going to use the same data as in Demo 5A. The only difference is that I'm going to use TF-IDF to extract features. Since we've gone over this process in 5A, we won't spend time on it here. Run this, and once it's finished, we will set up a K-Means model.

In [ ]:
# Import modules
import pandas as pd, numpy as np, json
from zipfile import ZipFile
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords


# Load data
with ZipFile('/storage/ice-shared/mgt8833/classdata/complaints.json.zip','r') as z:
    complaints = json.loads(z.read('complaints.json'))
    
complaintsDF = pd.DataFrame(complaints)
complaintsDF = complaintsDF.loc[complaintsDF['complaint_what_happened']!=""]

# Sample data
complaintsDF = complaintsDF.sample(10000,random_state=123)

# load stop words
stops = set(stopwords.words('english'))
stops.add("xxxx")
stops.add("xx/xx/xxxx")

# set up vectorizer
vec = TfidfVectorizer(token_pattern = r'\b[a-zA-Z]{3,}\b',
                     stop_words = list(stops),
                     ngram_range = (1,2),
                     max_features = 1000)

# generate DTM
dtm = vec.fit_transform(complaintsDF['complaint_what_happened'])

## K-Means Clustering with `sklearn`
We will use a batch-training version of K-Means clustering available through scikit-learn. Like with the LDA and NMF models we have used, training is very straightforward with `sklearn`'s consistent API. The specific model is called `MiniBatchKMeans`, and you can find the documentation for this model __[here](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.MiniBatchKMeans.html)__. Compared to LDA, there aren't quite as many parameters, and most of the parameters are related to the training process. Here are a the ones that we will focus on for now:

1. `n_clusters`: the number of clusters (recall that k-means requires us to set this)
2. `random_state`: to allow us all to get the same results
3. `reassignment_ratio`: relates to the how centroids are re-defined. In classic K-means, they are recomputed every iteration but in most faster implementations there's a tolerance that's evaluated.

Let's start with 20 clusters and evaluate the breakdown of cluster assignments:

In [ ]:
from sklearn.cluster import MiniBatchKMeans as KMeans

kmeans = KMeans(n_clusters = 20, random_state = 123, reassignment_ratio = 0.05)
labels = kmeans.fit_predict(dtm) # notice "predict" instead of "transform"
labels

Often, I'm interested in how distributed clusters are within the data. We can check this with `value_counts()` in `pandas`:

In [ ]:
pd.Series(labels).value_counts()

This suggests about 25% of the samples are in the two most populous clusters, and there are two with fewer than 200. We can look at a sample complaint in the most and least populous clusters by incorporating the labels into our original data. Let's take a look at complaints in the top and bottom two clusters

In [ ]:
complaintsDF['kmeans_label.1'] = labels
for cl in [19,0,10,6]:
    print(f"Cluster {cl}:\n")
    for complaint in complaintsDF.loc[complaintsDF['kmeans_label.1']==cl,'complaint_what_happened'].head(3):
        print(complaint)
        print("-----------------\n")

It's hard to make much sense of the themes in these clusters. Let's try a more analytic approach to try to understand what's driving cluster assignment.

### Evaluating Individual Clusters

Recall that clusters under k-means are spherical. This is easy to visualize in 2 or 3 dimensional space, but in 1,000 dimension space it's more difficult. This problem is impounded because we are dealing with sparse data--there are lots of 0s in the DTM. We have a few options:
1. We can evaluate which words appear with greatest prevalance in each cluster.
2. We can determine which cluster dimensions are most dissimilar from other clusters.
3. We can identify which reviews are closest to each cluster center.

We'll look at each of these.

#### 1. Word prevalence 
This first option is pretty easy to implement because we have a document term matrix that gives us word counts. We just need to subset the matrix by cluster, and then look at column sums to see which words have the largest (weighted) counts:

In [ ]:
sum(labels==10)

In [ ]:
dtm_dense = dtm.todense()
vocab = np.array(vec.get_feature_names_out())

for cl in range(20):
    mask = labels==cl # creates array of trues and falses
    subset = dtm_dense[mask,:] # mask chooses rows, : = all columns
    top5 = np.array(subset.sum(axis=0)).argsort()[0][-5:][::-1] # similar to how we identify indices for top words before (converted to array to simplify changing dimensions)
    print(f"Top 5 words for cluster {cl}: {'|'.join(vocab[top5])}")

This highlights an interesting issue: We see "wells fargo" showing up in in a cluster. Should complaints be organized by what organization is mentioned? This also happened in our LDA exercise. The answer likely depends on the goal. One way to address this is to replace all company/bank mentions with something generic, like "\_BANK\_". 

#### 2. Defining Features
For this method, we need to access the cluster centers themselves. We will then compare each dimension to the average dimensions across other clusters. The largest values (in absolute terms) should indicate dimensions, which correspond to words, that most uniquely define the cluster. An easy way to accomplish this is to standardize the centers to have mean 0 and then look for the largest absolute values (which equal distance from the mean).

Let's start by accessing and exploring the cluster centroids.

In [ ]:
centers = kmeans.cluster_centers_
print(centers.shape)

To standardize, we'll use `sklearn`'s `StandardScaler` (we'll discuss this more later):

In [ ]:
from sklearn.preprocessing import StandardScaler
standardizer = StandardScaler()
centers_std = standardizer.fit_transform(centers)
centers_std
#centers_std.mean(axis=0) # all 1k dimensions have mean 0 (effectively)
#centers_std.std(axis=0) # all 1k dimensions have STD 1

Next, convert this to absolute values, and then iterate over each cluster, identifying the (arguments of) the largest values with the row corresponding to that cluster:

In [ ]:
centers_abs = np.abs(centers_std)
for cl in range(20):
    inf_words = centers_abs[cl,:].argsort()[-5:][::-1]
    print(f"Top 5 words for cluster {cl}: {'|'.join(vocab[inf_words])}")

These seem similar, though maybe more specific than the ones identified in the first approach, probably because this method is not as affected by frequently occurring words since we standardized.

#### 3. Nearness to centroid
The K-means object we used can be used to directly access reviews closest to each cluster center. Recall that we used `fit_predict` when fitting the model. This method *predicts* the "label", or cluster membership, by choosing the closest cluster center. We can also use the `transform` method to provide the distance to each cluster center; the minimum of these distances will correspond to the observation's cluster assignment. We don't have to refit the model to access `transform`.

In [ ]:
distances = kmeans.transform(dtm)
distances.shape

The 10k by 20 shape is consistent with 20 distances per observation. Let's look at one row, find the minimum distance, find the reference/index of that index, and see if that coincides with the assigned cluster label:

In [ ]:
print(distances[0])
print(distances[0].min())
print(distances[0].argmin())
print(labels[0]) 

Now we can find the most representative complaint for each cluster by (1) subsetting `distances` for each cluster, (2) locating the minimum score for that cluster in that subset, and (3) accessing that complaint:

In [ ]:
for cl in range(20):
    subdist = distances[labels==cl,cl] # step 1: select rows and column needed
    model_idx = subdist.argmin() # step 2
    print(f"The complaint closest to the cluster center for {cl} is\n\n: {complaintsDF.iloc[model_idx]['complaint_what_happened']}") # step 3 (iloc = location based indexing)
    print("--------------------------------")

## Dimension Reduction, Standardizing, and Tuning K-means

Recall that clustering algorithms like K-means can be negatively impacted by features with varying scales and by the curse of high dimensionality. For scale, euclidean distance may be dominated by features with relatively larger scales, and in high dimensional data all observations look fairly dissimilar. 

We're next going to consider both **standardization** and **dimensionality reduction** as we try to tune the best K-means model. We'll start by looking at how to implement each technique individually, and then we'll incorporate all into a "grid-search" type approach which will faciliate model selection.

#### Standardization 
The most common means of standardizing data is to both "shift" (or center) and re-scale all data such that each feature has mean 0 and standard deviation 1. It's important to note that this method *retains all within-feature variation*. That is, we are not changing anything about the relative scale *within* each feature--if X1 for observation 1 is 10 times larger than X1 for observation 2, that difference will be retained. 

Rather, we are removing variation *across* features--X1, X2, and X3 all have the same scale. This is often a desirable preprocessing step. We've already seen how to standardize data with `sklearn`'s __[StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html)__, which is what we will use. Let's generate a standardized DTM:

In [ ]:
dtm_std = standardizer.fit_transform(np.asarray(dtm.todense())) # as of more recent versions. np.matrix doesn't work with many methods. must cast to array
dtm_std

In [ ]:
dtm_std.std(axis=0)

#### Dimension Reduction with PCA
Recall that principal component analysis, or PCA, identifies a set of common factors that explain the majority of the variation in the original data. There are no hard-and-fast rules when using PCA, but a common practice is to retain only those factors with eigenvalues greater than 1, or those factors that explain at least as much variation as an individual feature. You can also pick a target threshold of total variance that you want to explain. For instance, suppose we want to retain factors that explain at least 75% of the overall variance in the data. 

Scikit-learn makes it very easy to implement __[PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html)__ and to use any of these thresholds to select components. The implementation of PCA in `sklearn` can actually automatically select the number of components to retail using a few different criteria (see `n_components` in documentation), but we will manually select our components to retain:

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(svd_solver = 'full') # "full" = classic implementation using Singular Value Decomposition. There are other, faster approaches that approximate PCA for use with large data
temp = pca.fit_transform(np.asarray(dtm.todense()))
temp.shape

The specific eigenvectors are located in `components_`, and you can evaluate how much each vector explains with the `explained_variance_ratio`. 

In [ ]:
pca.explained_variance_ratio_[:20]

So the first component explains 3.4%. That may not seem like much, but keep in mind there are 1,000 features, so if all features were uncorrelated 1 feature would explain 0.1%. 

Often it's useful to evaluate the cumulative explained variance (the components are sorted most significant to least significant):

In [ ]:
pca.explained_variance_ratio_.cumsum()[:20]

Let's set a threshold at 75% and see how many components we'd have to retain:

In [ ]:
threshold = 0.75
(pca.explained_variance_ratio_.cumsum()<=threshold).sum() # So the first 357 components explain 3/4 of the total variance.

Let's generate a dataset with those 357 factors:

In [ ]:
dtm_pca = temp[:,pca.explained_variance_ratio_.cumsum()<=threshold]
dtm_pca.shape

If we had kept only those factors with eigenvalues exceeding 1, we would have kept fewer:

In [ ]:
(pca.explained_variance_ratio_>(1/1000)).sum()

Before we move to tuning, note that the PCA-based DTM likely also has scales that vary across components:

In [ ]:
pd.DataFrame(dtm_pca).describe().transpose()

It's unclear whether we want to retain that variation, so let's generate a 4th version of our DTM which is a standardized-PCA. We'll then save these four dataframes in a dictionary for reasons that will become clear shortly.

In [ ]:
dtm_pca_std = standardizer.fit_transform(dtm_pca)
all_dtms = {'original':dtm,
            'standardized':dtm_std,
            'pca':dtm_pca,
            'pca_std':dtm_pca_std}

#### Tuning the K-Means Model
We now have four different DTMs to consider. We'll try values for k that vary between 10 and 100 by 5s. For each iteration, we'll save the SSE (called `inertia`) and the mean silhouette score (quality of clusters):

In [ ]:
from sklearn.metrics import silhouette_score
records = []
models = []
for k in range(10,101,5):
    print(k)
    for label,d in all_dtms.items():
        print(label)
        record = {}
        record['dtm_type'] = label
        record['k'] = k
        kmeans = KMeans(n_clusters = k, random_state = 123, reassignment_ratio = 0.05, n_init = 'auto') # added n_init to avoid warnings in newer versions, if you receive an error - try removing it
        kmeans.fit(d)
        record['sse'] = kmeans.inertia_
        record['silhouette'] = silhouette_score(d,kmeans.predict(d))
        records.append(record)
        models.append(kmeans)

In [ ]:
diagDF = pd.DataFrame(records)

We can use `groupby` to get a sense of how the different DTMs perform. However, `inertia` will not be comparable since it depends on the scale of the input data

In [ ]:
diagDF.groupby('dtm_type')[['sse','silhouette']].mean()

Like with LDA, we can also plot these. Before doing so, though, we need to standardize the SSE within each group to have the same variance. We can do this with groupby and transform:

In [ ]:
diagDF.groupby('dtm_type')[['sse']].transform(lambda x: (x - x.mean())/x.std()) # must use transform to maintain order.

In [ ]:
diagDF['sse_std'] = diagDF.groupby('dtm_type')[['sse']].transform(lambda x: (x - x.mean())/x.std())

In [ ]:
diagDF

This time, we'll use seaborn to plot the data. Seaborn facilitates plotting summary statistics and makes it easy to assign "groups" like we have in this data. We can use `hue` to control color and `style`to vary the linestyle by group:

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

fig, axs = plt.subplots(2,1) # 2 rows, 1 column (so 2 plots)
sns.lineplot(x='k',y='sse_std',hue='dtm_type',style='dtm_type',data=diagDF,ax=axs[0])
sns.lineplot(x='k',y='silhouette',hue='dtm_type',style='dtm_type',data=diagDF,ax=axs[1])

## HDBSCAN and UMAP
Next, we'll examine how to cluster using __[HDBSCAN](https://hdbscan.readthedocs.io/en/latest/api.html)__ and reduce the dimensionality of data with __[UMAP](https://umap-learn.readthedocs.io/en/latest/)__

### Fitting a Basic Model

HDBSCAN is technically not part of `sklearn`, but it's implemented using essentially the same API. Let's fit a model with default values for all parameters to start:

In [ ]:
from hdbscan import HDBSCAN
hdb = HDBSCAN()
labels = hdb.fit_predict(dtm)

In [ ]:
pd.Series(labels).value_counts()

So what does this mean? -1 is the label given to "noise" points, meaning they aren't assigned to a cluster. This is a problem! Almost 80% of our data was labeled as "noise".

While HDBSCAN is pretty technologically sophisticated, it frankly does not work well with high-dimensional data. Let's try to reduce dimensions, this time with UMAP. 

Like HDBSCAN, UMAP is technically *not* part of scikit-learn. But, also like HDBSCAN, it's author(s) maintain an API that feels very comparable to scikit-learn. Because HDBSCAN is density-based, methods like UMAP are likely better suited than PCA because they focus on maintaining local and global relationships among data rather than string linear projections.

Here are a few key parameters in setting up UMAP:
- `n_neighbors`: This controls the number of "nearest neighbors" to use to projecting the "manifold approximation" (or the lower-dimension projection. Defaults to 15.
- `n_components`: The number of dimensions to project to. Defaults to 2 (think visualization).
- `metric`: how closeness is measured. Defaults to euclidean.

There are many other methods related to the fitting process and optimization. We'll leave those alone. We'll now project our DTM to 10 dimensions, leaving `n_neighbors` as the default. We'll use cosine similarity to measure closeness (`cosine`):

In [ ]:
from umap.umap_ import UMAP
um = UMAP(n_components = 10, metric = 'cosine', random_state=123)

reduced = um.fit_transform(dtm)

Let's check the output:

In [ ]:
reduced

In [ ]:
reduced.shape

Now we can try HDBSCAN again:

In [ ]:
labels2 = hdb.fit_predict(reduced)

In [ ]:
pd.Series(labels2).value_counts()

This seems better, but we still have (1) 30% of observations labeled as "noise", and (2) too many clusters!

### Tuning HDBSCAN
For brevity, we're going to focus only on tuning HDBSCAN, though you could also consider varying the number of components using with UMAP. We're going to vary these components:
- `cluster_selection_method`: `leaf` or `eom` (excess of mass). The former leads to more homogenous clusters, the latter more persistent.
- `min_cluster_size`: defaults to 5
- `min_samples`: establishes how many points needed to create "neighborhood"; defaults to None
- `cluster_selection_epsilon`: a threshold for creating new cluster. Points within this distance are merged into current cluster; defaults to 0

Let's run a variety of options with these four parameters. I'd like to collect the following information:
1. the number of points classified as noise
2. the number of clusters
3. the maximum cluster size
4. the standard deviation of cluster size
5. the number of minimum size clusters

I've set up the basics of the loop for you below. I'd like you to update "record" with the desired outputs listed above. **PAUSE** and try to add that code, and come back when you're done for the answer.

In [ ]:
records = []
for cluster_selection_method in ['eom','leaf']:
    input_params = {'cluster_selection_method':cluster_selection_method}
    for min_cluster_size in [5,25,100]:
        input_params['min_cluster_size'] = min_cluster_size
        for min_samples in [None,5,10]:
            input_params['min_samples'] = min_samples
            for cluster_selection_epsilon in [0,0.01,0.05,0.10]:
                input_params['cluster_selection_epsilon'] = cluster_selection_epsilon
                print(input_params)
                hdb = HDBSCAN(**input_params) # handy way to convert dictionary to keyword=...
                labels = hdb.fit_predict(reduced)
                record = input_params.copy() # creates a new object so we don't update input_params
                ## INSERT CODE BELOW ###
                

#### To summarize:
(1) HDBSCAN is a state of the art clustering method that is very efficient and flexible
(2) HDBSCAN struggles with high-dimensional data, but works well with UMAP
(3) HDBSCAN is difficult to tune (when starting with a raw DTM. It works better with embeddings).